In [23]:
from pyod.models.abod import ABOD
from pyod.models.knn import KNN

In [1]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import ipywidgets as widgets
%matplotlib inline


plt.rcParams['figure.figsize'] = (14,8)
#plt.rcParams['figure.dpi'] = 150
sns.set()
sns.set_context("talk")

**Import data**

In [2]:
repo_url = 'https://raw.githubusercontent.com/Xiaoqi-Sun/aptamer_scoring/main/'

# raw data
for i in np.arange(2,7):
    exec("R{}E = pd.read_csv(repo_url+'serotonin%20raw%20data/{}RE.csv')".format(i, i))
    exec("R{}C = pd.read_csv(repo_url+'serotonin%20raw%20data/{}RC.csv')".format(i, i))
    
# processed data
for i in np.arange(2,7):
    exec("R{}E_frequency = pd.read_csv(repo_url+'serotonin%20processed%20data/R{}E_frequency.csv',index_col='Quadrumer')".format(i, i))
    exec("R{}C_frequency = pd.read_csv(repo_url+'serotonin%20processed%20data/R{}C_frequency.csv',index_col='Quadrumer')".format(i, i))
    exec("R{}E_full_table_weighted = pd.read_csv(repo_url+'serotonin%20processed%20data/R{}E_full_table_weighted.csv',index_col=0)".format(i, i))
    exec("R{}C_full_table_weighted = pd.read_csv(repo_url+'serotonin%20processed%20data/R{}C_full_table_weighted.csv',index_col=0)".format(i, i))

In [3]:
# experimental dF/F values

# round 1: Sanghwa's 80 sequences
dFF = pd.read_csv(repo_url+'dFF%20data/dFF_r1.csv',usecols=[0,1,2])

# round 2: Xiaoqi's Prediction
dFF2_new = pd.read_csv(repo_url+'dFF%20data/dFF_r2_new.csv')
#dFF2 = pd.read_excel('dFF2.xlsx').loc[:,['df/f','Trimed']].rename(columns={'Trimed':'Sequence'})
#dFF2_old = pd.DataFrame({'Name':["N/A" for x in range(10)], 'Sequence':dFF2['Sequence'], 'dFF':dFF2['df/f']})

# round 3: Payam's prediction
#dFF3_old = pd.read_csv(repo_url+'dFF%20data/dFF_r3_old.csv',usecols=[0,1,3]).rename(columns={'dFF_1195':'dFF'}) #use dFF value at 1195nm
dFF3_new = pd.read_csv(repo_url+'dFF%20data/dFF_r3_new.csv',usecols=[0,1,3]).rename(columns={'dFF_1195':'dFF'}) #use dFF value at 1195nm

# round 4: Payam's  prediction (Model 3)
dFF4_pos = pd.read_csv(repo_url+'dFF%20data/dFF_r4_positive.csv')
dFF4_neg = pd.read_csv(repo_url+'dFF%20data/dFF_r4_negative.csv')


In [4]:
# general function for calculating quad score

def max_freq_ratio(quad_seq):
    #find the last term of the score definition
    r2=R2E_frequency[R2E_frequency.index==quad_seq]['Weighted frequency'][0] if quad_seq in R2E_frequency.index else 0
    r3=R3E_frequency[R3E_frequency.index==quad_seq]['Weighted frequency'][0] if quad_seq in R3E_frequency.index else 0
    r4=R4E_frequency[R4E_frequency.index==quad_seq]['Weighted frequency'][0] if quad_seq in R4E_frequency.index else 0
    r5=R5E_frequency[R5E_frequency.index==quad_seq]['Weighted frequency'][0] if quad_seq in R5E_frequency.index else 0      
    r6=R6E_frequency[R6E_frequency.index==quad_seq]['Weighted frequency'][0]
    
    # handle the inf case
    r6r5 = 0 if r5==0 else r6/r5
    r5r4 = 0 if r4==0 else r5/r4
    r4r3 = 0 if r3==0 else r4/r3
    r3r2 = 0 if r3==0 else r3/r2
    return max(r3r2,r4r3,r5r4,r6r5)

def max_freq_ratio_ctrl(quad_seq):
    #find the last term of the score definition
    r2=R2C_frequency[R2C_frequency.index==quad_seq]['Weighted frequency'][0] if quad_seq in R2C_frequency.index else 0
    r3=R3C_frequency[R3C_frequency.index==quad_seq]['Weighted frequency'][0] if quad_seq in R3C_frequency.index else 0
    r4=R4C_frequency[R4C_frequency.index==quad_seq]['Weighted frequency'][0] if quad_seq in R4C_frequency.index else 0
    r5=R5C_frequency[R5C_frequency.index==quad_seq]['Weighted frequency'][0] if quad_seq in R5C_frequency.index else 0      
    r6=R6C_frequency[R6C_frequency.index==quad_seq]['Weighted frequency'][0]
    
    # handle the inf case
    r6r5 = 0 if r5==0 else r6/r5
    r5r4 = 0 if r4==0 else r5/r4
    r4r3 = 0 if r3==0 else r4/r3
    r3r2 = 0 if r3==0 else r3/r2
    
    return max(r3r2,r4r3,r5r4,r6r5)

def extract_quadrumers(aptamer_sequence):
    #takes in one 18-mer and return a table of quadrumers, with a position column and a quadrumer column
    quadrumers = []
    for i in np.arange(15):
        quad = aptamer_sequence[i:i+4]
        quadrumers = np.append(quadrumers,quad)
    return quadrumers


def quad_score_exp(set_1_percentile, exp_ind_weight, name):
    #inputs: set_1_percentile -> see definition of set 1; {exp,ctrl}weight: weight for two indicator functions
    #return a dataframe with quadrumer as index and weighted frequency and scores as two columns
    #using quadrumers in R6E for calculation 
    
    #set 1: Kmers with frequencies once greater than 99.5th percentile of the kmers in the control round
    control_percentile = np.percentile(R2E_frequency['Weighted frequency'], set_1_percentile)
    set1 = R6E_frequency[R6E_frequency['Weighted frequency']>control_percentile]
    set1_list = set1.index

    #set 2: : with the same class size and consisting of kmers with the largest amplification-fold values was then defined. 
    set2 = R2E_frequency.merge(R6E_frequency,on='Quadrumer').rename(columns={'Weighted frequency_x':'R2E freq', 'Weighted frequency_y':'R6E freq'})
    set2['amp-fold value'] = set2['R6E freq']/set2['R2E freq']
    set2 = set2.sort_values('amp-fold value', ascending=False).head(len(set1))
    set2_list = set2.index
    
    score_r6 = []
    for i in R6E_frequency.index:
        term1 = (i in set1_list) or (i in set2_list)
        term2 = (i in set1_list)
        term3 = max_freq_ratio(i)
        score_r6 = np.append(score_r6, term1*exp_ind_weight + term2*exp_ind_weight + term3)
        
    R6E_with_score = R6E_frequency.copy()
    R6E_with_score[name+'_exp'] = score_r6
    return R6E_with_score

def quad_score_ctrl(set_1_percentile, ctrl_ind_weight, name):
    #return a dataframe with quadrumer as index and weighted frequency and scores as two columns
    #using quadrumers in R6E for calculation 
    
    #set 1 NOTE: using 99.5 percentile only gives 3 quadrumers
    control_percentile = np.percentile(R2C_frequency['Weighted frequency'],set_1_percentile) 
    set1 = R6C_frequency[R6C_frequency['Weighted frequency'] > control_percentile]
    set1_list = set1.index
    
    #set 2: : with the same class size and consisting of kmers with the largest amplification-fold values was then defined. 
    set2 = R2C_frequency.merge(R6C_frequency,on='Quadrumer').rename(columns={'Weighted frequency_x':'R2C freq', 'Weighted frequency_y':'R6C freq'})
    set2['amp-fold value'] = set2['R6C freq']/set2['R2C freq']
    set2 = set2.sort_values('amp-fold value', ascending=False).head(len(set1))
    set2_list = set2.index
    
    score_r6 = []
    for i in R6C_frequency.index:
        term1 = (i in set1_list) or (i in set2_list)
        term2 = (i in set1_list)
        term3 = max_freq_ratio_ctrl(i)
        score_r6 = np.append(score_r6, term1*ctrl_ind_weight + term2*ctrl_ind_weight + term3)
        
    R6C_with_score = R6C_frequency.copy()
    R6C_with_score[name+'_ctrl'] = score_r6
    return R6C_with_score

def quad_score_full(set_1_percentile_exp, set_1_percentile_ctrl, exp_ind_weight, ctrl_ind_weight, exp_weight, ctrl_weight, name):
    quad_exp = quad_score_exp(set_1_percentile_exp, exp_ind_weight, name)
    quad_ctrl = quad_score_ctrl(set_1_percentile_ctrl, ctrl_ind_weight, name)
    
    merged = quad_exp.merge(quad_ctrl, how='left', left_index=True, right_index=True)
    
    merged[name] = exp_weight*merged[name+ '_exp'] - ctrl_weight*merged[name+'_ctrl']
    return pd.DataFrame({'Weighted frequency': merged['Weighted frequency_x'], #weighted frequence is from R6E
                        name : merged[name] })

In [5]:
# general functions for calculating aptamer score 

def aptamer_score(RnE, quad_score, name):
    # Returns a dataframe like R6E, with 18-mer sequence and score for each aptamer
    # Inputs: RnE: a dataframe with 18-mer sequences;
    #        quad_score: a df , out put of quad_score_full function,
    
    quadrumer_score = quad_score
    score_name = quadrumer_score.columns[1]
    
    aptamer_score = []
    aptamer_freqsum = []
    for apt_seq in RnE['Trimed']:
        all_quads = extract_quadrumers(apt_seq)
        one_score = 0
        one_freqsum = 0
        for quad in all_quads:
            if len(quadrumer_score[quadrumer_score.index==quad]) != 0:
                one_score += quadrumer_score.loc[quad][1]
                one_freqsum += quadrumer_score.loc[quad][0]
        aptamer_score = np.append(aptamer_score, one_score)
        aptamer_freqsum = np.append(aptamer_freqsum, one_freqsum)
        
    tbl_with_score = RnE.copy().loc[:,['Trimed']]
    tbl_with_score[name]=aptamer_score
    tbl_with_score['Weighted frequency'] = aptamer_freqsum
    tbl_with_score.index = tbl_with_score.index + 1 # reset index to match!
    
    
    tbl_with_score[name+' percent'] = 100*tbl_with_score[name]/max(tbl_with_score[name])
    tbl_with_score[name+' su'] = (tbl_with_score[name]-np.mean(tbl_with_score[name]))/np.std(tbl_with_score[name])


    return tbl_with_score


def aptamer_score_dFF(dFF_tbl, quad_score, name):
    # for incorporating all 80 sequences of dFF table
    #Returns a dataframe like R6E, with 18-mer sequence and score for each aptamer
    #Inputs: RnE: a dataframe with 18-mer sequences;
    #        quad_score: a df , out put of quad_score_full function,
    quadrumer_score = quad_score
    score_name = quadrumer_score.columns[1]
    
    aptamer_score = []
    aptamer_freqsum = []
    for apt_seq in dFF_tbl['Sequence']:
        all_quads = extract_quadrumers(apt_seq)
        one_score = 0
        #one_freqsum = 0
        for quad in all_quads:
            if len(quadrumer_score[quadrumer_score.index==quad]) != 0:
                one_score += quadrumer_score.loc[quad][1]
                #one_freqsum += quadrumer_score.loc[quad][0]
        aptamer_score = np.append(aptamer_score, one_score)
        
    tbl_with_score = dFF_tbl.copy().loc[:,['Name','Sequence','dFF']]
    tbl_with_score[name]=aptamer_score
    tbl_with_score.index = tbl_with_score.index + 1 # reset index to match!
    
    
    tbl_with_score[name+' percent'] = 100*tbl_with_score[name]/max(tbl_with_score[name])
    tbl_with_score[name+' su'] = (tbl_with_score[name]-np.mean(tbl_with_score[name]))/np.std(tbl_with_score[name])


    return tbl_with_score

def dFF_with_score(threshold, dFF_tbl, quad_score, score_name):
    dFF_with_score = aptamer_score_dFF(dFF_tbl, quad_score, name=score_name)
    dFF_with_score['Y/N'] = dFF_with_score['dFF'].map(lambda x: 'Y' if x>threshold else 'N')
    return dFF_with_score

# Scorings

$$
Score_{quad,exp} ={\color{red}{exp\,ind\,weight}} * I(quad\in Set1\,|\,quad\in Set2) + {\color{red}{exp\,ind\,weight}} * I(quad\in Set1) + max(\frac{freq_{quad,j+1}}{freq_{quad}}|j\in R)
$$
$$
Score_{quad,ctrl} ={\color{blue}{ctrl\,ind\,weight}} * I(quad\in Set1\,|\,quad\in Set2) + {\color{blue}{ctrl\,ind\,weight}} * I(quad\in Set1) + max(\frac{freq_{quad,j+1}}{freq_{quad}}|j\in R)
$$

$$
Score_{quad,final} = {\color{brown}{exp\,weight}}*Score_{quad,exp} - {\color{green}{ctrl\,weight}}*Score_{quad,ctrl}
$$


Set1: Kmers with frequencies once greater than 99.5th percentile of the kmers in the control round

Set 2: with the same class size and consisting of kmers with the largest amplification- fold values was then defined.

In [6]:
from ipywidgets import interact_manual, interactive

In [7]:
@interact_manual(exp_percentile= widgets.FloatSlider(min=90,max=100,step=0.5,value=99.5,style={'description_width': '100px'}),
                 ctrl_percentile = widgets.FloatSlider(min=90,max=100,step=0.5,value=99.5,style={'description_width': '100px'}),
                 exp_ind_weight = widgets.IntSlider(min=1,max=10,value=1,style={'description_width': '100px'}),
                 ctrl_ind_weight = widgets.IntSlider(min=1,max=10,value=1,style={'description_width': '100px'}),
                 exp_weight = widgets.IntSlider(min=1,max=10,value=1,style={'description_width': '100px'}),
                 ctrl_weight = widgets.IntSlider(min=1,max=10,value=1,style={'description_width': '100px'}),
                 name=widgets.Text(value='score name'),
                 )

def f(exp_percentile, ctrl_percentile, exp_ind_weight, ctrl_ind_weight, exp_weight, ctrl_weight, name):
    
    threshold = 1.3
    temp_quads_score = quad_score_full(exp_percentile, ctrl_percentile, exp_ind_weight, ctrl_ind_weight,exp_weight, ctrl_weight, name)
    
    dFF_1_with_score_tbl = dFF_with_score(threshold, dFF_tbl=dFF, quad_score=temp_quads_score, score_name=name)
    dFF_2_with_score_tbl = dFF_with_score(threshold, dFF_tbl=dFF2_new, quad_score=temp_quads_score, score_name=name)
    dFF_3_with_score_tbl = dFF_with_score(threshold, dFF_tbl=dFF3_new, quad_score=temp_quads_score, score_name=name)
    dFF_4P_with_score_tbl = dFF_with_score(threshold, dFF_tbl=dFF4_pos, quad_score=temp_quads_score, score_name=name)
    dFF_4N_with_score_tbl = dFF_with_score(threshold, dFF_tbl=dFF4_neg, quad_score=temp_quads_score, score_name=name)
    
    dFF_1_with_score_tbl['Round'] = ['Round 1' for x in range(len(dFF_1_with_score_tbl))]
    dFF_2_with_score_tbl['Round'] = ['Round 2' for x in range(len(dFF_2_with_score_tbl))]
    dFF_3_with_score_tbl['Round'] = ['Round 3' for x in range(len(dFF_3_with_score_tbl))]
    dFF_4P_with_score_tbl['Round'] = ['Round 4P' for x in range(len(dFF_4P_with_score_tbl))]
    dFF_4N_with_score_tbl['Round'] = ['Round 4N' for x in range(len(dFF_4N_with_score_tbl))]
    
    dFF_combined = dFF_1_with_score_tbl.append(dFF_2_with_score_tbl).append(dFF_3_with_score_tbl).append(dFF_4P_with_score_tbl).append(dFF_4N_with_score_tbl)
    
    # outlier detection
#     X_train = dFF_with_score_tbl.loc[:,['dFF',name]].values  #change training set, doesn't train on scores in su
#     clf_list = [ABOD(),KNN()]

#     for clf in clf_list:
#         outlier_label = clf.fit(X_train).predict(X_train)
#         dFF_with_outlier = dFF_with_score_tbl.copy()
#         dFF_with_outlier['outlier'] = outlier_label
#         outliers_dropped = dFF_with_outlier[dFF_with_outlier['outlier']==0]
#         temp_r = outliers_dropped.corr().iloc[0,3]
#         print('round 1 only r for ',str(clf)[:4],' is', temp_r)
        
    # calculate Pearson correlation coefficient
    #    position [0,1] is the correlation between raw score with dFF values
    #    position [0,3] is the correlation between score in su with dFF values -> wrong, shouldn't put Y and Xsu together

    print('round 1 only correlation coefficent for all sequences is ', dFF_1_with_score_tbl.corr().iloc[0,1])
    print('round 2 only correlation coefficent for all sequences is ', dFF_2_with_score_tbl.corr().iloc[0,1])
    print('round 3 only correlation coefficent for all sequences is ', dFF_3_with_score_tbl.corr().iloc[0,1])
    print('r1 and r2 correlation coefficent for all sequences is ', dFF_1_with_score_tbl.append(dFF_2_with_score_tbl).corr().iloc[0,1])
    print('overall correlation coefficent for all sequences is ', dFF_combined.corr().iloc[0,1])
    
    # ploting
    plt_title = "{n} for R6E 18-mers, T={th}, exp_ind_w = {exp_ind_w}, ctrl_ind_w = {ctrl_ind_w}, exp_w = {exp}, ctrl_w={ctrl}".format(n=name,th=threshold, exp_ind_w=exp_ind_weight,ctrl_ind_w=ctrl_ind_weight,exp =exp_weight,ctrl=ctrl_weight)
    
    # update this line for plot in different scales
    fig = px.scatter(dFF_combined, y="dFF", x=name, symbol="Y/N", color='Round', 
                     hover_name="Name", hover_data=["Sequence", "dFF", name])

    
    fig.update_layout(title_text=plt_title)
    fig.show()
    

interactive(children=(FloatSlider(value=99.5, description='exp_percentile', min=90.0, step=0.5, style=SliderSt…

## Generate CSV  
  
Run the following cell and select desired parameters, then click run interact. The csv file will appear in the same folder that this notebook is located.  
Output csv file has 4 columns: 
- raw score (X-axis)
- dFF (Y-axis)
- Round (label)
- Name (for reference only)

In [8]:
@interact_manual(exp_ind_weight = widgets.IntSlider(min=1,max=20,value=1,style={'description_width': '100px'}),
                 ctrl_ind_weight = widgets.IntSlider(min=1,max=20,value=1,style={'description_width': '100px'}),
                 exp_weight = widgets.IntSlider(min=1,max=20,value=1,style={'description_width': '100px'}),
                 ctrl_weight = widgets.IntSlider(min=1,max=20,value=1,style={'description_width': '100px'}),
                 )

def f(exp_ind_weight, ctrl_ind_weight, exp_weight, ctrl_weight):
    
    temp_quads_score = quad_score_full(99.5, 99.5, exp_ind_weight, ctrl_ind_weight,exp_weight, ctrl_weight, 'raw score')
    
    dFF_1_with_score_tbl = dFF_with_score(threshold=1.5, dFF_tbl=dFF, quad_score=temp_quads_score, score_name='raw score')
    dFF_2_with_score_tbl = dFF_with_score(threshold=1.5, dFF_tbl=dFF2_new, quad_score=temp_quads_score, score_name='raw score')
    dFF_3_with_score_tbl = dFF_with_score(threshold=1.5, dFF_tbl=dFF3_new, quad_score=temp_quads_score, score_name='raw score')
    dFF_4P_with_score_tbl = dFF_with_score(threshold=1.5, dFF_tbl=dFF4_pos, quad_score=temp_quads_score, score_name='raw score')
    dFF_4N_with_score_tbl = dFF_with_score(threshold=1.5, dFF_tbl=dFF4_neg, quad_score=temp_quads_score, score_name='raw score')
    
    dFF_1_with_score_tbl['Round'] = ['Round 1' for x in range(len(dFF_1_with_score_tbl))]
    dFF_2_with_score_tbl['Round'] = ['Round 2' for x in range(len(dFF_2_with_score_tbl))]
    dFF_3_with_score_tbl['Round'] = ['Round 3' for x in range(len(dFF_3_with_score_tbl))]
    dFF_4P_with_score_tbl['Round'] = ['Round 4P' for x in range(len(dFF_4P_with_score_tbl))]
    dFF_4N_with_score_tbl['Round'] = ['Round 4N' for x in range(len(dFF_4N_with_score_tbl))]
    
    dFF_combined = dFF_1_with_score_tbl.append(dFF_2_with_score_tbl).append(dFF_3_with_score_tbl).append(dFF_4P_with_score_tbl).append(dFF_4N_with_score_tbl)
    
    # dFF_combined has columns : Name, Sequence, dFF, raw score, raw score percent, raw score su, Y/N, Round
    
    result = dFF_combined.loc[:,['raw score', 'dFF', 'Round','Name']]
    file_name = "E_ind={exp_ind_w}, C_ind={ctrl_ind_w}, E_w={exp}, C_w={ctrl}".format(exp_ind_w=exp_ind_weight,ctrl_ind_w=ctrl_ind_weight,exp =exp_weight,ctrl=ctrl_weight)
    
    result.to_csv(file_name+'.csv')

interactive(children=(IntSlider(value=1, description='exp_ind_weight', max=20, min=1, style=SliderStyle(descri…